In [ ]:
import datetime
from abc import ABC, abstractmethod

# UTILITÁRIOS

def mensagem(msg):
    print("[BANCO LOPES]", msg)

# MODELOS

class Historico:
    def __init__(self):
        self.transacoes = []

    def adicionar_transacao(self, tipo, valor):
        self.transacoes.append({
            "tipo": tipo,
            "valor": valor,
            "data": datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        })

    def mostrar(self):
        if not self.transacoes:
            return "Nenhuma transação realizada."
        return "\n".join([f"{t['data']} - {t['tipo']}: R$ {t['valor']:.2f}" for t in self.transacoes])

class Conta:
    def __init__(self, agencia, numero, cliente):
        self.agencia = agencia
        self.numero = numero
        self.cliente = cliente
        self.saldo = 0.0
        self.historico = Historico()

    def depositar(self, valor):
        if valor > 0:
            self.saldo += valor
            self.historico.adicionar_transacao("Depósito", valor)
            mensagem(f"Depósito de R$ {valor:.2f} realizado com sucesso.")
            return True
        mensagem("Valor inválido para depósito.")
        return False

    def sacar(self, valor):
        if 0 < valor <= self.saldo:
            self.saldo -= valor
            self.historico.adicionar_transacao("Saque", valor)
            mensagem(f"Saque de R$ {valor:.2f} realizado com sucesso.")
            return True
        mensagem("Saque não autorizado. Verifique saldo e limite.")
        return False

    def extrato(self):
        extrato = self.historico.mostrar()
        extrato += f"\nSaldo atual: R$ {self.saldo:.2f}\n"
        return extrato

class ContaCorrente(Conta):
    def __init__(self, agencia, numero, cliente, limite_saque=500, limite_saques_dia=3):
        super().__init__(agencia, numero, cliente)
        self.limite_saque = limite_saque
        self.limite_saques_dia = limite_saques_dia
        self.saques_realizados = 0
        self.data_ultimo_saque = None

    def sacar(self, valor):
        hoje = datetime.date.today()
        if self.data_ultimo_saque != hoje:
            self.saques_realizados = 0
            self.data_ultimo_saque = hoje

        if self.saques_realizados >= self.limite_saques_dia:
            mensagem("Limite diário de saques atingido.")
            return False

        if valor > self.limite_saque:
            mensagem(f"Valor excede o limite de saque: R$ {self.limite_saque:.2f}.")
            return False

        if super().sacar(valor):
            self.saques_realizados += 1
            return True
        return False

class Cliente:
    def __init__(self, nome, cpf, nascimento, endereco):
        self.nome = nome
        self.cpf = cpf
        self.nascimento = nascimento
        self.endereco = endereco
        self.contas = []

    def adicionar_conta(self, conta):
        self.contas.append(conta)

    def recuperar_conta(self):
        return self.contas[0] if self.contas else None

class Banco:
    def __init__(self):
        self.clientes = []
        self.contas = []
        self.numero_conta = 1

    def buscar_cliente(self, cpf):
        return next((c for c in self.clientes if c.cpf == cpf), None)

    def criar_cliente(self, nome, cpf, nascimento, endereco):
        if self.buscar_cliente(cpf):
            mensagem("CPF já cadastrado.")
            return
        cliente = Cliente(nome, cpf, nascimento, endereco)
        self.clientes.append(cliente)
        mensagem("Cliente cadastrado com sucesso.")

    def listar_clientes(self):
        if not self.clientes:
            mensagem("Nenhum cliente cadastrado.")
            return
        print("\n--- LISTA DE CLIENTES ---")
        for c in self.clientes:
            print(f"Nome: {c.nome} | CPF: {c.cpf} | Nascimento: {c.nascimento} | End: {c.endereco}")

    def criar_conta_corrente(self, cpf):
        cliente = self.buscar_cliente(cpf)
        if not cliente:
            mensagem("Cliente não encontrado.")
            return
        conta = ContaCorrente("0001", str(self.numero_conta), cliente)
        cliente.adicionar_conta(conta)
        self.contas.append(conta)
        self.numero_conta += 1
        mensagem("Conta criada com sucesso.")

    def depositar(self, cpf, valor):
        cliente = self.buscar_cliente(cpf)
        if not cliente:
            mensagem("Cliente não encontrado.")
            return
        conta = cliente.recuperar_conta()
        if not conta:
            mensagem("Conta não encontrada.")
            return
        conta.depositar(valor)

    def sacar(self, cpf, valor):
        cliente = self.buscar_cliente(cpf)
        if not cliente:
            mensagem("Cliente não encontrado.")
            return
        conta = cliente.recuperar_conta()
        if not conta:
            mensagem("Conta não encontrada.")
            return
        conta.sacar(valor)

    def ver_extrato(self, cpf):
        cliente = self.buscar_cliente(cpf)
        if not cliente:
            mensagem("Cliente não encontrado.")
            return
        conta = cliente.recuperar_conta()
        if not conta:
            mensagem("Conta não encontrada.")
            return
        print("\n--- EXTRATO ---")
        print(conta.extrato())

    def listar_contas(self):
        if not self.contas:
            mensagem("Nenhuma conta cadastrada.")
            return
        print("\n--- LISTA DE CONTAS ---")
        for conta in self.contas:
            print(f"Agência: {conta.agencia} | Conta: {conta.numero} | Cliente: {conta.cliente.nome} | Saldo: R$ {conta.saldo:.2f}")

    def solicitar_financiamento(self, cpf, tipo):
        cliente = self.buscar_cliente(cpf)
        if not cliente:
            mensagem("Cliente não encontrado.")
            return
        conta = cliente.recuperar_conta()
        if not conta:
            mensagem("Conta não encontrada.")
            return
        try:
            valor = float(input(f"Digite o valor do financiamento de {tipo}: "))
            parcelas = int(input("Digite o número de parcelas: "))
            if valor > 0 and parcelas > 0:
                parcela_mensal = valor / parcelas
                conta.historico.adicionar_transacao(f"Financiamento {tipo}", valor)
                mensagem(f"Financiamento {tipo} aprovado!\nValor: R$ {valor:.2f} em {parcelas}x de R$ {parcela_mensal:.2f}.")
            else:
                mensagem("Dados inválidos para financiamento.")
        except ValueError:
            mensagem("Entrada inválida. Use apenas números.")

banco = Banco()


def main():
    while True:
        print("""
╔════════════════════════════════════╗
║       SISTEMA BANCÁRIO LOPES       ║
╠════════════════════════════════════╣
║ 1. Criar Cliente                   ║
║ 2. Criar Conta                     ║
║ 3. Depositar                       ║
║ 4. Sacar                           ║
║ 5. Ver Extrato                     ║
║ 6. Listar Contas                   ║
║ 7. Listar Clientes                 ║
║ 8. Financiamento Imóvel            ║
║ 9. Financiamento Veículo           ║
║ 10. Empréstimo Pessoal             ║
║ 0. Sair                            ║
╚════════════════════════════════════╝
""")
        op = input("Escolha uma opção: ").strip()
        if op == "1":
            nome = input("Nome: ").strip()
            cpf = input("CPF: ").strip()
            nasc = input("Data de nascimento: ").strip()
            end = input("Endereço: ").strip()
            banco.criar_cliente(nome, cpf, nasc, end)
        elif op == "2":
            cpf = input("CPF do cliente: ").strip()
            banco.criar_conta_corrente(cpf)
        elif op == "3":
            cpf = input("CPF do cliente: ").strip()
            try:
                valor = float(input("Valor para depósito: "))
                banco.depositar(cpf, valor)
            except ValueError:
                mensagem("Entrada inválida. Use apenas números.")
        elif op == "4":
            cpf = input("CPF do cliente: ").strip()
            try:
                valor = float(input("Valor para saque: "))
                banco.sacar(cpf, valor)
            except ValueError:
                mensagem("Entrada inválida. Use apenas números.")
        elif op == "5":
            cpf = input("CPF do cliente: ").strip()
            banco.ver_extrato(cpf)
        elif op == "6":
            banco.listar_contas()
        elif op == "7":
            banco.listar_clientes()
        elif op == "8":
            cpf = input("CPF do cliente: ").strip()
            banco.solicitar_financiamento(cpf, "Imóvel")
        elif op == "9":
            cpf = input("CPF do cliente: ").strip()
            banco.solicitar_financiamento(cpf, "Veículo")
        elif op == "10":
            cpf = input("CPF do cliente: ").strip()
            banco.solicitar_financiamento(cpf, "Empréstimo Pessoal")
        elif op == "0":
            print("Saindo...")
            break
        else:
            mensagem("Opção inválida!")

if __name__ == "__main__":
    main()



╔════════════════════════════════════╗
║       SISTEMA BANCÁRIO LOPES       ║
╠════════════════════════════════════╣
║ 1. Criar Cliente                   ║
║ 2. Criar Conta                     ║
║ 3. Depositar                       ║
║ 4. Sacar                           ║
║ 5. Ver Extrato                     ║
║ 6. Listar Contas                   ║
║ 7. Listar Clientes                 ║
║ 8. Financiamento Imóvel            ║
║ 9. Financiamento Veículo           ║
║ 10. Empréstimo Pessoal             ║
║ 0. Sair                            ║
╚════════════════════════════════════╝

[BANCO LOPES] Cliente cadastrado com sucesso.

╔════════════════════════════════════╗
║       SISTEMA BANCÁRIO LOPES       ║
╠════════════════════════════════════╣
║ 1. Criar Cliente                   ║
║ 2. Criar Conta                     ║
║ 3. Depositar                       ║
║ 4. Sacar                           ║
║ 5. Ver Extrato                     ║
║ 6. Listar Contas                   ║
║ 7. Listar Cli